In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


In [2]:
# Image preprocessing: resize, convert to tensor, normalize
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
}


In [4]:
data_dir = "archive/dataset"

train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(root=f"{data_dir}/test", transform=data_transforms['test'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# Load pretrained MobileNetV3
model = models.mobilenet_v3_small(pretrained=True)  # or mobilenet_v3_large

# Replace the classifier with the number of classes in your dataset
num_classes = len(train_dataset.classes)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = model.to(device)


/home/fatweeb/SIH/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fatweeb/SIH/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /home/fatweeb/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100.0%


Using device: cuda


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
num_epochs = 5  # adjust as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")


Epoch 1/5, Loss: 0.2188, Accuracy: 91.58%
Epoch 2/5, Loss: 0.1656, Accuracy: 93.80%
Epoch 3/5, Loss: 0.1444, Accuracy: 94.72%
Epoch 4/5, Loss: 0.1254, Accuracy: 95.40%
Epoch 5/5, Loss: 0.1067, Accuracy: 96.06%


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

# ----------------------------
# 1. Config
# ----------------------------
data_dir = "archive/dataset"  # change if needed
batch_size = 32
num_epochs = 5
learning_rate = 0.001
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

# ----------------------------
# 2. Data transforms
# ----------------------------
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}

# ----------------------------
# 3. Datasets & loaders
# ----------------------------
train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'test'), transform=data_transforms['test'])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

num_classes = 2
print(f"Detected classes: {train_dataset.classes}")

# ----------------------------
# 4. Model setup
# ----------------------------
model = models.mobilenet_v3_small(pretrained=True)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
model = model.to(device)

# ----------------------------
# 5. Loss and optimizer
# ----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ----------------------------
# 6. Training loop with tqdm
# ----------------------------
best_acc = 0.0  # track best test accuracy
best_model_path = "mobilenetv3_best.pth"

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")
    for inputs, labels in loop:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        loop.set_postfix(loss=running_loss/len(loop), acc=100*correct/total)
    
    # ----------------------------
    # Evaluate on test set after each epoch
    # ----------------------------
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total_test += labels.size(0)
            correct_test += predicted.eq(labels).sum().item()
    
    test_acc = 100 * correct_test / total_test
    print(f"Test Accuracy after epoch {epoch+1}: {test_acc:.2f}%")
    
    # Save best model
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"Best model saved with accuracy: {best_acc:.2f}%\n")
    
    model.train()  # back to training mode


Detected classes: ['O', 'R']


/home/fatweeb/SIH/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fatweeb/SIH/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch [1/5]: 100%|██████████| 706/706 [00:42<00:00, 16.55it/s, acc=91.6, loss=0.22]  


Test Accuracy after epoch 1: 91.64%
Best model saved with accuracy: 91.64%



Epoch [2/5]: 100%|██████████| 706/706 [00:41<00:00, 17.16it/s, acc=93.8, loss=0.163] 


Test Accuracy after epoch 2: 92.96%
Best model saved with accuracy: 92.96%



...

Test Accuracy after epoch 3: 92.08%


Epoch [4/5]: 100%|██████████| 706/706 [00:42<00:00, 16.66it/s, acc=95.6, loss=0.123] 


Test Accuracy after epoch 4: 92.92%


Epoch [5/5]: 100%|██████████| 706/706 [00:42<00:00, 16.63it/s, acc=95.9, loss=0.108] 


Test Accuracy after epoch 5: 88.78%


In [5]:
import torch
from torchvision import transforms, models
from PIL import Image
# Use the same model architecture
model = models.mobilenet_v3_small(pretrained=False)
model.classifier[3] = torch.nn.Linear(model.classifier[3].in_features, 2)  # 2 classes
model.load_state_dict(torch.load("mobilenetv3_best.pth", map_location='cpu'))  # use 'cuda' if GPU
model.eval()  # important: set to evaluation mode

# Transform to match training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                         [0.229, 0.224, 0.225])
])

# Load image
image_path = "test_3.jpg"  # path to your image
image = Image.open(image_path).convert("RGB")  # ensure 3 channels
image = transform(image).unsqueeze(0)  # add batch dimension

with torch.no_grad():
    outputs = model(image)
    _, predicted = outputs.max(1)  # class index: 0 or 1

print(f"Predicted class index: {predicted.item()}")


class_names = ['organic', 'recyclable']
print(f"Predicted class: {class_names[predicted.item()]}")


Predicted class index: 1
Predicted class: recyclable
